In [ ]:
#Importing the python libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import sklearn as sk 
import os 
import cv2
import albumentations as A
from PIL import Image 
from tensorflow import keras 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 

#Setting the training and testing paths to extract the files 
training_path = "/kaggle/input/sp-society-camera-model-identification/train/train/"
testing_path = "/kaggle/input/sp-society-camera-model-identification/test/test/"
_,training_classes,_=next(os.walk(training_path))
#print(training_classes)


In [ ]:
#Setting the hyperparameters for the deep learning model
batch_size = 8
epochs = 40
learning_rate = 1e-4

In [ ]:
#testing_files 
_,_,testing_files = next(os.walk(testing_path))


In [ ]:
#Creating the training dataset by iterating over the directory 
#Appending the training paths to the list and extracting the classes from the directory
training_paths = [] 
labels = [] 
for i in training_classes:
    _,_,filenames = next(os.walk(training_path+i))
    for j in filenames:
        training_paths.append(training_path+i+'/'+j)
        labels.append(i)

In [ ]:
#Ensure that the training size and the class labels are the same
assert len(training_paths)==len(labels)

In [ ]:
#Setting the random seed to 33 for reproducible results
seed = 33

In [ ]:
#Moving the training paths to an pandas DataFrame for easy indexing and one-hot encoding the class labels
training_data = pd.DataFrame(training_paths,columns=['Training Image Path'])
classes = pd.DataFrame(labels)
classes = pd.get_dummies(classes)
classes.columns=np.unique(labels)# One Hot Encode the class variables

In [ ]:
#Binary encoding the classes for the second learning task in the neural network
binary_classification =  []
for i in labels:
    if(i=='Sony-NEX-7'):
        binary_classification.append('DSLR')
    else:
        binary_classification.append('Mobile')
        
set(binary_classification)

binary_classification = pd.DataFrame(binary_classification)
print(binary_classification[0].value_counts())
binary_classification.loc[binary_classification[0]=='Mobile']=0
binary_classification.loc[binary_classification[0]=='DSLR']=1
print(binary_classification[0].value_counts())

In [ ]:
#print the training data
training_data

In [ ]:
#printing the one-hot encoded class variable
classes.head()

In [ ]:
#Creating the class dictionary for converting the one-hot encoded variables
classes_dict = {'0':'HTC-1-M7',
                '1':'LG-Nexus-5x',
                '2':'Motorola-Droid-Maxx',
                '3':'Motorola-Nexus-6',
                '4':'Motorola-X',
                '5':'Samsung-Galaxy-Note3',
                '6':'Samsung-Galaxy-S4',
                '7':'Sony-NEX-7',
                '8':'iPhone-4s',
                '9':'iPhone-6'}

In [ ]:
#Creating the testing DataFrame for easy indexing
testing_data = pd.DataFrame(testing_files,columns=['Testing Image Path'])
testing_data

In [ ]:
#Reading images form the disk to memory and converting from BGR to RGB
def read_img(path):
    temp = cv2.imread(path)
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
    return np.array(temp)

In [ ]:
#Defining the training and testing augmentations
length = 512 #Defining the length used for each image 
training_transforms = A.Compose([A.RandomCrop(height=length,width=length),
                                A.RandomGamma(gamma_limit=(80,120),p=0.9),
                                A.JpegCompression(quality_lower=70,quality_upper=90,p=0.9),
                                A.GridDistortion(interpolation=cv2.INTER_CUBIC)])
    

                                
testing_augmentation = A.Compose([
                                  A.CenterCrop(height=length,width=length)])

In [ ]:
#Creating a custom keras dataset to create mini-batch arrays that are fed to the neural network
class Dataset(keras.utils.Sequence):
    def __init__(self,x,y,y_binary,batch_size,augmentations,test):
        self.x = x
        self.y = y
        self.y_binary = y_binary
        self.batch_size = batch_size
        self.augmentations = augmentations 
        self.test = test 
    def __len__(self):
        return int(len(self.x)/self.batch_size)
    def __getitem__(self,index):
        batched_x = self.x.iloc[index*self.batch_size:(index+1)*self.batch_size].to_numpy()
        if(self.test):
            images = [(self.augmentations(image=read_img(i))['image']) for i in batched_x]
            
        else:
            batched_y = self.y.iloc[index*self.batch_size : (index+1)*self.batch_size,:]
            batched_y_binary = self.y_binary.iloc[index*self.batch_size : (index+1)*self.batch_size]
            images = [(self.augmentations(image=read_img(i[0]))['image']) for i in batched_x]
        return np.array(images)if self.test else (np.array(images),[np.array(batched_y.values),np.array(batched_y_binary.values)])

In [ ]:
#Creating a dummy datast to test the functionality
dummy_dataset = Dataset(training_data,classes,binary_classification,batch_size=batch_size,augmentations=training_transforms,test=False)

In [ ]:
#Calling the dummy dataset to check the functionality 
(a,b)=dummy_dataset.__getitem__(5)
print(a[0])
print(a.shape)
print(b[1])
print(type(b[1]))
plt.imshow(a[0])

losses  = keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
losses_1 = keras.losses.BinaryCrossentropy(label_smoothing=0.1)

In [ ]:
#Creating the Keras model for Multi-tasking and learning two things at once
def keras_model(length,input_size,output_classes):
        p=0.3
        base_model = tf.keras.applications.EfficientNetB4(weights='imagenet', include_top=False, input_shape=[length, length, 3])
        base_model.trainable = False
        inputs =keras.layers.Input(shape=(length,length,3))
        x = keras.applications.efficientnet.preprocess_input(inputs)
        x = base_model(x, training=False)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(1024,activation='relu')(x)
        x = keras.layers.Dense(256,activation='relu')(x)
        x = keras.layers.Dense(64,activation='relu')(x)
        outputs = keras.layers.Dense(10, activation='softmax')(x)
        outputs1 = keras.layers.Dense(1,activation='sigmoid',kernel_regularizer=keras.regularizers.L2(0.01))(x)
        model =keras.Model(inputs=inputs, outputs=[outputs,outputs1])

        model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate), loss=[losses,losses_1], metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])

        model.summary() 
        return model

In [ ]:
#Instantiating the keras model
training_model = keras_model(length,length*length*3,10)

In [ ]:
# Splitting the training dataset into a training and validation dataset with the validation split  = 0.1 and setting the Shuffle=True
xtrain,xval,ytrain,yval = train_test_split(training_data,classes,test_size=0.1,shuffle=True,random_state=seed)

In [ ]:
#Creating the binary label for the validation and training dataset
ybinary_train = binary_classification.iloc[xtrain.index].astype(int)
ybinary_val = binary_classification.iloc[xval.index].astype(int)
assert len(ybinary_train)==len(ytrain)
assert len(ybinary_val)==len(yval)

print(ybinary_train.info())

In [ ]:
#Printing the size of each array to check that the sizes match
print(len(xtrain),len(xval))
print(len(ytrain),len(yval))

In [ ]:
#Pushing the arrays into a custom keras dataset to feed to the neural networ
training_dataset = Dataset(xtrain,ytrain,ybinary_train,batch_size=batch_size,augmentations=training_transforms,test=False)
validation_dataset = Dataset(xval,yval,ybinary_val,batch_size=1,augmentations=training_transforms,test=False)

print(len(training_dataset))
print(len(validation_dataset))

In [ ]:
#Creating a list of callbacks to add to the training model 

model_file = "output/base_model_weights.hd5" # defining the output path to save the model file

checkpoint = tf.keras.callbacks.ModelCheckpoint(model_file, monitor="val_accuracy", save_best_only=True, mode='max') # Save the model which has the best validation acc

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.9, patience=2, min_lr=1e-6, mode="max", verbose=True) # reduce Lr if the validation acc does not increase

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5, mode="max", verbose=True) # Stop the training if the val acc does not improve

callbacks_list = [checkpoint, reduce_lr, early_stopping] # Call all of the instantiated callbacks into one variable


In [ ]:
#training the model 
training_model.fit(training_dataset,validation_data=validation_dataset,epochs=epochs,batch_size=batch_size,callbacks=callbacks_list,verbose=1)


In [ ]:
#Reading the submission file from the directory 
sample_submission = pd.read_csv('/kaggle/input/sp-society-camera-model-identification/sample_submission.csv')
sample_submission.head() #Printing the top few rows from the submission file

In [ ]:
#Creating the testing dataset
x_test = testing_path+sample_submission['fname']
print(x_test[0])

x_test = Dataset(x_test,x_test,x_test,test=True,batch_size=1,augmentations=testing_augmentation)

In [ ]:
#generating predictions for the validation dataset
val_predicted = training_model.predict(validation_dataset,verbose=1)

In [ ]:
##reading the best file from memory and generating predictions for the testing dataset
predicted = training_model.predict(x_test,verbose=1)

print(predicted)

In [ ]:
#Creating a function to convert the one-hot encoded item into class labels and splitting the two predictions
def separate_the_two(array):
    return array[0],array[1]


def convert_predictions_to_labels(labels):
    temp = []
    for item in labels.argmax(axis=1):
        temp.append(classes_dict[str(item)])
    return temp



In [ ]:
#Calling the function to separate the two predictions
validation_labels,binary_val_labels=separate_the_two(val_predicted)
test_labels,test_binary_labels=separate_the_two(predicted)

In [ ]:
#Converting the one hot encoded values to labels
test_labels = convert_predictions_to_labels(test_labels)

validation_labels = convert_predictions_to_labels(validation_labels)

In [ ]:
#filling up the submission file with the test predictions
sample_submission['camera'] = test_labels
sample_submission.head()

In [ ]:
#Submitting the test predictions
sample_submission.to_csv("submission.csv", index=False)

In [ ]:
#Converting the validation one-hot encoded labels
yval=yval.idxmax(axis=1)

In [ ]:
#printing the confusion matrix
con_matrix = confusion_matrix(yval,validation_labels)
print(con_matrix)

In [ ]:
#Creating a function to convert raw predictions to binary values
def convert_from_binary(array):
    temp = []
    for i in array:
        if(i>=0.5):temp.append(1)
        else:temp.append(0)
    return temp

In [ ]:
#Calling the binary conversion function for the validation data
binary_val_labels = convert_from_binary(binary_val_labels)

In [ ]:
#printing the confusion matrix for the binary classification problem
binary_con_matrix = confusion_matrix(ybinary_val,binary_val_labels)
print(binary_con_matrix)